In [1]:
import subprocess
import os
import ast
import time
import pdb

def run_linux_command(command):
    p = subprocess.Popen(command, universal_newlines=True, shell=True, 
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    text = p.stdout.read()
    retcode = p.wait()
    return text

def remove_kubernetes_administrative_containers(dict_containers_nameid):
    copy_dict_containers_nameid = dict_containers_nameid.copy()
    kubernetes_aministrative_containers = ['k8s_POD', 'k8s_sidecar', 'k8s_dnsmasq', 'k8s_kubernetes', 'k8s_storage-provisioner', 'k8s_kube', 'k8s_etcd']
    for container_name in kubernetes_aministrative_containers:
        for container_id in dict_containers_nameid.keys():
            if dict_containers_nameid[container_id].startswith(container_name):
                try:
                    del copy_dict_containers_nameid[container_id]
                except:
                    pass
    return copy_dict_containers_nameid

def get_container_names(path):
    import yaml
    filenames = os.listdir(path)
    filenames = [filename for filename in filenames if filename.endswith('.yml')]
    container_created_name = []
    for filename in filenames:
        with open(filename, 'r') as stream:
            try:
                yaml_content = (yaml.load(stream))
                if yaml_content['kind'] == 'Deployment':
                    container_created_name.append(yaml_content['spec']['template']['spec']['containers'][0]['name'])
            except yaml.YAMLError as exc:
                pass #print(exc)

    return(container_created_name)

def containerid_from_deployedcontainer(deployed_containers):
    Allcontainers_running = run_linux_command('docker ps --format "{{.ID}}: {{.Names}}"').split('\n')[:-1]
    Allcontainers_nameid = []
    for details in Allcontainers_running:
        Allcontainers_nameid.append(details.split(':'))

    deployed_containerid = []
    for name in deployed_containers:
        for record in Allcontainers_nameid:
            if name in record[1]:
                deployed_containerid.append(record[0])

    return deployed_containerid  

path = os.getcwd()
run_linux_command('kubectl create -f bitcoin.yml')
run_linux_command('kubectl create -f bitcoin2.yml')
print 'Bitcoin deployment created.'

run_linux_command('kubectl create -f deep_learning.yml')
print 'Deep learning depoyment created. '

#run_linux_command('kubectl create -f mysql-pv.yml')
#run_linux_command('kubectl create -f mysql-service.yml')
#run_linux_command('kubectl create -f mysql-deployment.yml')
#print 'Mysql deployment created.'

#run_linux_command('kubectl create -f stress.yml')
#print 'Stress Ubuntu workload created.'

print 'Waiting for 30 sec to let workload run.'
time.sleep(30)  #Wait for 30 sec to receive bitcoin task from main server

#print subprocess.check_output(["docker", "ps", "--format", "table {{.ID}} \t {{.Names}} \t {{.Status}}"])
#os.system('git clone https://github.com/cloudviz/agentless-system-crawler')
current_working_dir = os.getcwd()
#new_path = current_working_dir+'/agentless-system-crawler'
#os.chdir(new_path)
#os.system('docker build -t crawler .')
#Image_build =  subprocess.check_output(['docker', 'image', 'ls'])
#for i,crawl in enumerate(Image_build.split('\n')):
#    if i == 0:
#        print crawl
#    if 'crawler' in crawl:
#        print crawl

os.chdir(path)
deployed_containers = get_container_names(os.getcwd())
deployed_containerid = containerid_from_deployedcontainer(deployed_containers)
        
crawler_command = ['docker', 'run', '--rm', '--privileged', '--net=host', '--pid=host', '-v', '/cgroup:/cgroup:ro', '-v', '/sys/fs/cgroup:/sys/fs/cgroup:ro', '-v', '/var/lib/docker:/var/lib/docker:ro', '-v', '/var/run/docker.sock:/var/run/docker.sock', '-v', '-it', 'crawler', '--crawlmode', 'OUTCONTAINER', '--features', 'cpu,metric,process,connection', '--crawlContainers', ','.join(deployed_containerid)]
crawler_output = subprocess.check_output(crawler_command)
crawler_output = crawler_output.split('\n')

os.chdir(current_working_dir)
crawler_record_linestart = []
containers_scanned = []
dict_containers_nameid = {}

for i,records in enumerate(crawler_output):
    if records.startswith('metadata'):
        crawler_record_linestart.append(i)
        container_id =  ast.literal_eval(records.split('\t')[2:][0])['container_short_id']
        containers_scanned.append(container_id)
        dict_containers_nameid[container_id] = ast.literal_eval(records.split('\t')[2:][0])['container_name']

dict_containers_nameid = remove_kubernetes_administrative_containers(dict_containers_nameid)
print 'Containers scanned are : ',dict_containers_nameid

Bitcoin deployment created.
Deep learning depoyment created. 
Waiting for 30 sec to let workload run.
Containers scanned are :  {'fe26b8b8e242': 'k8s_monero-cpuminer-deployment_monero-cpuminer-deployment-bb5fb4dd9-kn92t_default_9b32d7ef-a791-11e8-a186-025271d58932_0', '5c368c1f7c4f': 'k8s_deep-learning-dynamic_deeplearning-deployment-7c7c877959-st4b9_default_9b7bdf87-a791-11e8-a186-025271d58932_0'}


In [2]:
#Delete the informaton collected for crawler container itself
try:
    for i,records in enumerate(crawler_output):
        if records.startswith('metadata'):
            metadata_dict = ast.literal_eval(records.split('\t')[2:][0])
            container_image =  metadata_dict['docker_image_short_name']
            if 'crawler' in container_image:
                crawler_container = metadata_dict['container_short_id']
                crawler_container_index = containers_scanned.index(crawler_container)
                containers_scanned.remove(crawler_container)
                del dict_containers_nameid[crawler_container]
    try:
        del crawler_output[crawler_record_linestart[crawler_container_index] : crawler_record_linestart[crawler_container_index+1]]

    except:
        del crawler_output[crawler_record_linestart[crawler_container_index] : len(crawler_output)]


    print 'Length of collected crawler records:',len(crawler_output)
except:
    pass

In [3]:
crawler_record_linestart = []
containers_scanned = []

for i,records in enumerate(crawler_output):
    if records.startswith('metadata'):
        crawler_record_linestart.append(i)
        container_id =  ast.literal_eval(records.split('\t')[2:][0])['container_short_id']
        containers_scanned.append(container_id)
        
print(crawler_record_linestart,containers_scanned)

([0, 7, 13], ['fe26b8b8e242', '5c368c1f7c4f', 'fdd276453842'])


In [4]:
#Parse the CPU, process, metric, connection feature
dict_container_crawled = {}
cpu_names = []
process_names = []
connection_names = []
metric_names = []
for line_index,line_num,container_id in zip(range(len(crawler_record_linestart)),crawler_record_linestart,containers_scanned):
    for i in range(line_num+1,len(crawler_output)):
        try:
            if i >= crawler_record_linestart[line_index+1]:
                break
        except:
            pass
        if crawler_output[i].startswith('cpu'):
            cpu_names.append(crawler_output[i].split('\t')[2]) 
        
        if crawler_output[i].startswith('process'):    
            process_names.append(crawler_output[i].split('\t')[1])
            
        if crawler_output[i].startswith('metric'):    
            metric_names.append(crawler_output[i].split('\t')[2])
            
        if crawler_output[i].startswith('connection'):    
            connection_names.append(crawler_output[i].split('\t')[1])
    
    dict_container_crawled[container_id+'_cpu'] = cpu_names
    cpu_names = []
    dict_container_crawled[container_id+'_process'] = process_names
    process_names = []
    dict_container_crawled[container_id+'_metric'] = metric_names
    metric_names = []
    dict_container_crawled[container_id+'_connection'] = connection_names
    connection_names = []
            


In [5]:
dict_container_crawled

{'5c368c1f7c4f_connection': [],
 '5c368c1f7c4f_cpu': ['{"cpu_idle":39.122984,"cpu_nice":0.0,"cpu_user":57.65016027960673,"cpu_wait":0.0,"cpu_system":3.226855720393263,"cpu_interrupt":0.0,"cpu_steal":0.0,"cpu_util":60.877016}'],
 '5c368c1f7c4f_metric': ['{"cpupct":49.9,"mempct":32.12,"pname":"python","pid":1,"read":162164736,"rss":2688700416,"status":"sleeping","user":"root","vms":4005437440,"write":23224320}',
  '{"cpupct":0.0,"mempct":0.36,"pname":"python2.7","pid":340,"read":0,"rss":30208000,"status":"running","user":"root","vms":151310336,"write":0}'],
 '5c368c1f7c4f_process': ['"python/1"', '"python2.7/342"'],
 'fdd276453842_connection': [],
 'fdd276453842_cpu': ['{"cpu_idle":100.0,"cpu_nice":0.0,"cpu_user":0.0,"cpu_wait":0.0,"cpu_system":0.0,"cpu_interrupt":0.0,"cpu_steal":0.0,"cpu_util":0.0}'],
 'fdd276453842_metric': ['{"cpupct":0.0,"mempct":0.0,"pname":"pause","pid":1,"read":0,"rss":4096,"status":"sleeping","user":"0","vms":1048576,"write":0}',
  '{"cpupct":0.0,"mempct":0.36,"p

In [6]:
#processing CPU feature for kubernetes with dockersats
dict_container_CPUusage = {}
containers_scanned = dict_containers_nameid.keys()
for container_name in containers_scanned:
    CPU_sample_value = []
    for i in [0,1,2,3,4]:
        command = "docker stats " + container_name + " --no-stream | awk 'FNR == 2 {print $3}'"
        text = run_linux_command(command)
        CPU_sample_value.append(float(text.split('%')[0]))
        time.sleep(0.1)
    print max(CPU_sample_value)
    dict_container_CPUusage[container_name] = max(CPU_sample_value)
    
suspicious_containers = []

for dict_key in  sorted(dict_container_CPUusage.keys()):
    if dict_container_CPUusage[dict_key] > 30:
        print 'Alert! container: {0} might be suspicious'.format(dict_key)
        print 'The details are: \n', run_linux_command('docker ps --filter id='+dict_key)
        suspicious_containers.append(dict_key)

144.4
65.04
Alert! container: 5c368c1f7c4f might be suspicious
The details are: 
CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS               NAMES
5c368c1f7c4f        5026a6d335fa        "nohup python DEN/DE…"   14 minutes ago      Up 14 minutes                           k8s_deep-learning-dynamic_deeplearning-deployment-7c7c877959-st4b9_default_9b7bdf87-a791-11e8-a186-025271d58932_0

Alert! container: fe26b8b8e242 might be suspicious
The details are: 
CONTAINER ID        IMAGE                COMMAND                  CREATED             STATUS              PORTS               NAMES
fe26b8b8e242        bitnn/alpine-xmrig   "./xmrig -o stratum+…"   14 minutes ago      Up 14 minutes                           k8s_monero-cpuminer-deployment_monero-cpuminer-deployment-bb5fb4dd9-kn92t_default_9b32d7ef-a791-11e8-a186-025271d58932_0



In [7]:
from sets import Set
if len(Set(dict_container_CPUusage.keys()) - Set(dict_containers_nameid.keys())) == 0: 
    for key in dict_containers_nameid.keys():
        dict_container_CPUusage[dict_containers_nameid[key]] = dict_container_CPUusage[key]
        del dict_container_CPUusage[key]

    import matplotlib.pyplot as plt, mpld3
    lists = sorted(dict_container_CPUusage.items()) # sorted by key, return a list of tuples

    x, y = zip(*lists) # unpack a list of pairs into two tuples

    ax =plt.figure().add_subplot(111)
    plt.plot(x,y, 'ks-', mec='w', mew=5, ms=20)
    plt.xticks(x, sorted(dict_container_CPUusage.keys()), rotation=90)
    ax.set_xlabel('Containers')
    ax.set_ylabel('CPU Usage (%)')
    plt.savefig('CPU_plot.pdf')
    plt.show()
    #mpld3.display()
else:
    print 'Something is Wrong'

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: the sets module is deprecated
  """Entry point for launching an IPython kernel.


<Figure size 640x480 with 1 Axes>

In [8]:
#Showing the result in table
import pandas as pd
import numpy as np
df = pd.DataFrame.from_dict(dict_container_CPUusage, orient='index' )
df.columns = ['CPU Usage']
df['Syscalls'] = np.zeros((len(dict_container_CPUusage.keys()),1))

print df

                                                    CPU Usage  Syscalls
k8s_deep-learning-dynamic_deeplearning-deployme...      65.04       0.0
k8s_monero-cpuminer-deployment_monero-cpuminer-...     144.40       0.0


In [9]:
import fileinput
import re
import glob

def grep(PAT, FILES):
    returned_grep = []
    for line in fileinput.input(glob.glob(FILES)):
        if re.search(PAT, line):
             returned_grep.append(line)
    return returned_grep

In [17]:
suspicious_containers = ['5c368c1f7c4f', '6fc471847760', 'f185c48b505b']

In [18]:
import time
from multiprocessing import Process, Pipe
import sys
import pandas as pd
import pickle
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import tree


def collect_syscalls(mpPipe):
    # This will take a long time
    print "Syscalls collection started."
    with open('syscall_run_cmd.sh','w') as fl:
        fl.write("perf record -e 'raw_syscalls:sys_enter' -g --exclude-perf -a  --call-graph dwarf,4096 -o sys_call_collected -G kubepods/" + qosClass_type + "/pod"+poduid+ "/"+ container_longID)
    os.system('sh syscall_run_cmd.sh')
    mpPipe.send("Done")
    
def n_grams_split(input_list, len_ngram):
        n_gram_seq = []
        for i in range(len(input_list)-len_ngram+1):
            n_gram_seq.append(input_list[i:i+len_ngram])
        return(n_gram_seq)
    
for container_name in suspicious_containers:
    container_longID = subprocess.check_output(['docker', 'inspect', '--format="{{.Id}}"', container_name])
    container_longID = ast.literal_eval(container_longID[:-1])
    pod_name = run_linux_command("kubectl get pod -o jsonpath='{range .items[?(@.status.containerStatuses[].containerID==\"docker://" + container_longID +"\")]}{.metadata.name}{end}'")
    run_linux_command('kubectl get pods '+pod_name + ' -o yaml > pod_yamlfile')
    qosClass_type = grep('qosClass','pod_yamlfile')
    poduid = grep('uid','pod_yamlfile')
    os.system('rm -rf pod_yamlfile')
    qosClass_type = ''.join(qosClass_type)
    qosClass_type = qosClass_type.lower()
    if 'besteffort' in qosClass_type:
        qosClass_type = 'besteffort'
    else:
        qosClass_type = 'burstable'
    poduid = poduid[1].split(':')[1].strip() 
    #with open('syscall_run_cmd.sh','w') as fl:
    #    fl.write("perf record -e 'raw_syscalls:sys_enter' -g --exclude-perf -a  --call-graph dwarf,4096 -o sys_call_collected -G docker/" + container_longID)
    #os.system('sh syscall_run_cmd.sh')
    parent_conn, child_conn = Pipe()
    p = Process(target=collect_syscalls, args=(child_conn,))
    p.start()
    time.sleep(3)
    try:
        os.system('pkill -f perf')
    except:
        pass
    time.sleep(5)
    print "Syscalls collection completed."
    os.system('perf script -i sys_call_collected > sys_call_numbers')
    
    os.system('rm -rf syscall_run_cmd.sh')
    os.system('rm -rf sys_call_collected')
    
    with open('sys_call_numbers', 'r') as fl:
        content = fl.readlines()
    os.system('rm -rf sys_call_numbers')
    if len(content) > 5:
        Syscalls_list = []
        for i in range(len(content)):
            try: 
                Syscalls_list.append((content[i].split('NR')[1]).split(' ')[1])

            except:
                pass

        Syscalls_frame = pd.DataFrame(Syscalls_list)

        #Creating the sequence from the dataframe
        window_size = 10
        Sequence_anomaly = Syscalls_frame[Syscalls_frame.columns[0]]
        Sequence_anomaly = list(Sequence_anomaly)
        #Sequence_normal = split_list(Sequence_normal,window_size)
        Sequence_anomaly = n_grams_split(Sequence_anomaly,window_size)
        anomaly_seq_frame = pd.DataFrame(Sequence_anomaly)
        if len(Sequence_anomaly[-1]) < window_size:
            anomaly_seq_frame = anomaly_seq_frame.drop(anomaly_seq_frame.index[len(anomaly_seq_frame)-1])
            print 'Last anomaly sequence deleted.'
        X = anomaly_seq_frame.values
        grad_boost = pickle.load(open('grad_boosting.pkl', 'rb'))
        Y_pred_gradboost = grad_boost.predict(X)

        decision_tree = pickle.load(open('decision_tree.pkl', 'rb'))
        Y_pred_decisiontree = grad_boost.predict(X)

        match_count = 0
        unmatch_count = 0
        for i in range(len(X)):
            if Y_pred_gradboost[i] == Y_pred_decisiontree[i]:
                match_count+=1
            else:
                unmatch_count+=1

        if 0.8*match_count > unmatch_count:
            ML_prediction_correctness = True
        else:
            ML_prediction_correctness = False

        anomaly_predict_sample_gradboost = list(Y_pred_gradboost).count(1)
        normal_predict_sample_gradboost = list(Y_pred_gradboost).count(0)
        anomaly_predict_sample_decisiontree = list(Y_pred_decisiontree).count(1)
        normal_predict_sample_decisiontree = list(Y_pred_decisiontree).count(0)

        if 0.6*anomaly_predict_sample_gradboost > normal_predict_sample_gradboost:
            grad_boost = 'Anomaly'
        else:
            grad_boost = 'Normal'

        if 0.6*anomaly_predict_sample_decisiontree > normal_predict_sample_decisiontree:
            decision_tree = 'Anomaly'
        else:
            decision_tree = 'Normal'

        if grad_boost == 'Anomaly' and decision_tree == 'Anomaly' and ML_prediction_correctness == True:
            print 'Container {0} is anomaly, so disabling.'.format(container_name)
            print 'The details are: \n', run_linux_command('docker ps --filter id='+container_name)
            long_container_id = run_linux_command('docker inspect --format="{{.Id}}" '+ container_name)
            long_container_id = long_container_id[:-1]
            pod_name = run_linux_command("kubectl get pod -o jsonpath='{range .items[?(@.status.containerStatuses[].containerID==\"docker://" + long_container_id +"\")]}{.metadata.name}{end}'")
            run_linux_command('kubectl delete pods '+pod_name)
            print 'Disabled and deleted.'
        else:
            print 'Container {0} is found real and normal, alert was false.'.format(container_name)
            print 'The details are: \n', run_linux_command('docker ps --filter id='+container_name)
    else:
        print "syscal not generated for pod {0}, so skipping it.".format(pod_name)

Syscalls collection started.
Syscalls collection completed.
Container 5c368c1f7c4f is found real and normal, alert was false.
The details are: 
CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS               NAMES
5c368c1f7c4f        5026a6d335fa        "nohup python DEN/DE…"   About an hour ago   Up About an hour                        k8s_deep-learning-dynamic_deeplearning-deployment-7c7c877959-st4b9_default_9b7bdf87-a791-11e8-a186-025271d58932_0

Syscalls collection started.
Syscalls collection completed.
Container 6fc471847760 is anomaly, so disabling.
The details are: 
CONTAINER ID        IMAGE                COMMAND                  CREATED             STATUS              PORTS               NAMES
6fc471847760        bitnn/alpine-xmrig   "./xmrig -o stratum+…"   24 minutes ago      Up 24 minutes                           k8s_monero-cpuminer-deployment_monero-cpuminer-deployment-bb5fb4dd9-mc8qt_default_7f374133-a795-11e8

In [1]:
print run_linux_command('kubectl delete -f bitcoin.yml')
print run_linux_command('kubectl delete -f bitcoin2.yml')
print run_linux_command('kubectl delete -f deep_learning.yml')
print run_linux_command('kubectl delete -f mysql-pv.yml')
print run_linux_command('kubectl delete -f mysql-service.yml')
print run_linux_command('kubectl delete -f mysql-deployment.yml')
print run_linux_command('kubectl delete -f stress.yml')

NameError: name 'run_linux_command' is not defined